# Some examples

## Code

### Function concerning first examples: any decompositions as rank-one bilinear map

In [1]:
"""
Translate a set of indices into an element of the vector space `W`.

EXAMPLE:
    b(0, 1, V, n, m) gives the element representing a₀b₁ in V. 
    b([1, 0], [1, 1], V, n, m) gives the element representing a₀(b₀+b₁) in V. 
"""

def _b(I, J, W, n, m):
    B = W.basis()
    if I in ZZ and J in ZZ:
        d = I + J*n
        return B[d]
    else:
        S = W()
        for i in range(n):
            a = I[i]
            if a == 0:
                continue
            for j in range(m):
                b = J[j]
                if b == 0:
                    continue
                else:
                    d = i + j*n
                    S += (a*b) * B[d]
        return S

"""
Internal function needed to compute rank-one elements.
"""
def _construct_recc(L, begin, length, F):
    if length == 0:
        L.append(begin)
    else:
        for x in F:
            construct_recc(L, begin+[x], length-1, F)

"""
Internal function needed to compute rank-one elements.
"""
def _construct(length, F):
    L = []
    for j in range(length):
        construct_recc(L, j*[F(0)]+[F(1)], length-1-j, F)
    return L

"""
Compute the rank-one elements in V.
"""
def rank_one_elements(V, n, m):
    return [_b(x, y, V, n, m) for x in _construct(n, V.base_field()) for y in construct(m, V.base_field())]

"""
Compute the rank-one symmetric elements in V.
"""
def rank_one_elements_sym(V, n):
    return [_b(x, x, V, n, n) for x in _construct(n, V.base_field())]

In [2]:
"""
Compute a basis of W containing only rank-one elements.
"""
def rank_one_basis(W, G):
    
    B = [w for w in W.basis() if w in G]
    if len(B) == W.dimension():
        return B
    else:
        for g in [x for x in G if x in W]:
            if g not in W.span(B):
                B += [g]
        return B

In [3]:
"""
The `expand_subspace` as described in Covanov paper. Compute a vector space generated by rank-one elements that 
contains the elements in `targets`.

NOTE: v0
"""
def expand_subspace(targets, n, m, W = None, k = None, upto = Infinity, G = None, G2 = None, L=[]):
    if k <= upto:
        T = targets[0].parent().span(targets)

        if W == None:
            W = T

        if G == None:
            G = rank_one_elements(W.ambient_vector_space(), n, m)
            G2 = copy(G)

        if k == None:
            k = T.dimension()

        if W.dimension() == k and W.span([x for x in G if x in W]).dimension() == k:
            L.append(W)
        else:

            for g in G2:
                if g in W:
                    continue
                G3 = copy(G2)
                G3.remove(g)
                expand_subspace(targets, n, m, W+W.span([g]), k+1, upto, G, G3, L)

"""
The `expand_subspace` as described in Covanov paper. Compute a vector space generated by rank-one elements that 
contains the elements in `targets`.

NOTE: v1
"""                
def expand_subspace1(targets, n, m, W = None, k = None, upto = Infinity, G = None, G2 = None, L=[]):
    if k <= upto:
        T = targets[0].parent().span(targets)

        if W == None:
            W = T

        if G == None:
            G = rank_one_elements(W.ambient_vector_space(), n, m)
            G2 = copy(G)

        if k == None:
            k = T.dimension()

        if W.dimension() == k and W.span([x for x in G if x in W]).dimension() == k:
            L.append(W)
        else:

            for i in range(0, len(G2)):
                g = G2[i]
                if g in W:
                    continue
                G3 = copy(G2[i+1:])
                expand_subspace1(targets, n, m, W+W.span([g]), k+1, upto, G, G3, L)
                
"""
The `expand_subspace` as described in Covanov paper. Compute a vector space generated by rank-one elements that 
contains the elements in `targets`.

NOTE: v2
"""                 
def expand_subspace2(targets, n, m, W = None, elem = None, k = None, upto = Infinity, G = None, G2 = None, L=[]):
    
    if k <= upto:
        T = targets[0].parent().span(targets)

        if W == None:
            W = T

        if G == None:
            G = rank_one_elements(W.ambient_vector_space(), n, m)
            G2 = copy(G)

        if k == None:
            k = T.dimension()

        if W.dimension() == k and W.span([x for x in G if x in W]).dimension() == k:
            L.append(W)
        elif G2 != []:
            if elem == None:
                G2 = reduction(G2, W)
            else:
                G2 = reduction(G2, W.span([elem]))
            
            for i in range(len(G2)):
                g = G2[i]
                G3 = copy(G2[i+1:])
                expand_subspace2(targets, n, m, W+W.span([g]), g, k+1, upto, G, G3, L)

"""
Compute formulas to decompose the elements in `T` in rank-one elements. 
"""                  
def formulas(T, W, G):
    B = rank_one_basis(W, G)
    WW = W.span_of_basis(B)
    return B, [(t, WW.coordinates(t)) for t in T]

In [4]:
"""
Reduction of the elements in `G` modulo `W`.
""" 
def reduction(G, W):
    H = []
    B = W.echelonized_basis()
    M = W.echelonized_basis_matrix()
    P = M.pivots()
    for g in G:
        w = g.list_from_positions(P)
        h = g
        for j in range(len(w)):
            h -= w[j] * B[j]
        H.append(h)
    
    H.sort()
    I = [H[0]]
    for j in range(1, len(H)):
        if H[j-1] == H[j]:
            continue
        else:
            I.append(H[j])
    if I[0] == 0:
        I.pop(0)
    return I

"""
Reduction of the elements in `G` modulo `W`.
""" 
def reduction2(G, W):
    l = len(G)
    delete = []
    for i in range(l):
        for j in range(i+1, l):
            if (G[i]-G[j]) in W:
                delete.append(j)
    
    G2 = copy([G[i] for i in range(l) if i not in delete])
    return G2

### Function concerning tri-symmetric decomposition

In [5]:
"""
Compute a list `L` of lists of up to `upto` elements in `B` whose associated trace
bilinear form span a vector space containing `t`.
""" 
# À changer !!! On peut sûrement faire moins de calculs
def find_decomposition(t, B, upto, d, L, k = 0, G = [], W = None):
    if k <= upto:
        if W == None:
            W = t.parent().span([])
        if t in W:
            L.append(G)
        else:
            for j in range(len(B)):
                B2 = B[j+1:]
                G2 = copy(G)+[B[j]]
                find_decomposition(t, B2, upto, d, L, k+1, G2, W+t.parent().span([d[B[j]]]))

"""
Compute the bilinear map associated to `x`, represented by an element in `V`.
""" 
def trace_to_bil(x, V):
    n = sqrt(V.dimension())
    g = x.parent().gen()
    L = [(x * g^j).trace() for j in range(n)]
    return _b(L, L, V, n, n)

"""
Compute a dictionnary associating elements of `k` with their bilinear form in `V`.
""" 
def make_dict_global(k, V):
    d = dict()
    for x in k:
        d[x] = trace_to_bil(x, V)
    return d

"""
Compute the elements in `k` whose `j`-th coordinate in the canonical basis is 1, and the i-th coordinates
for i<j are zero.
""" 
def fields_elements(k, j):
    L = []
    for x in k:
        y = x.polynomial().coefficients(sparse=False)
        if (len(y) >= j+1) and (y[j] == 1):
            boo = True
            for l in range(j):
                if y[l] != 0:
                    boo = False
                    break
            if boo:
                L.append(x)
    return L

In [6]:
"""
Returns True if `T` is a list of zero elements. False otherwise.
""" 
def finished(T):
    for i in range(len(T)):
        if T[i] != 0:
            return False
    return True

"""
Compute the tri-symmetric decompositions of `T` 

NOTE:
    Apply the `find_decomposition` strategy in each coordinate of `T` searching for solutions with only 
    `upto_each` rank-one elements, while `upto` rank-one elements are allowed in the whole process.
""" 
def tri_symmetric_search(T, j, k, V, d, upto, upto_each, Lglob, Lloc = []):
    if finished(T):
        Lloc.sort()
        Lglob.append(Lloc)
    else:
        u = min(upto_each, upto) # how many products we can still use
        M = []
        find_decomposition(T[j], fields_elements(k, j), u, d, M)
        if len(M) > 0:
            for m in M:
                B = [d[s] for s in m]
                W = V.subspace_with_basis(B)
                coord = W.coordinates(T[j])
                S = copy(T)
                Lloc2 = copy(Lloc)
                for i in range(len(coord)):
                    Lloc2.append((m[i], coord[i]))
                    P = m[i].polynomial()
                    for l in range(j, P.degree()+1):
                        S[l] = S[l] - P[l]*coord[i]*d[m[i]]
                tri_symmetric_search(S, j+1, k, V, d, upto-len(m), upto_each, Lglob, Lloc2)        

In [7]:
"""
Compute the list of vector of `V` associated with the multiplication bilinear form in `k`.
""" 
def multiplication_formula(k, V):
    P = k.modulus()
    n = k.degree()
    T = (2*n-1)*[V()]
    for j in range(n):
        for i in range(n):
            T[i+j] += _b(j, i, V, n, n)
    
    l = len(T)
    
    while l > n:
        for j in range(P.degree()+1):
            T[l-1-n+j] -= P[j]*T[l-1]
        l -= 1
        T.pop(-1)
    
    return T

### Functions concerning Frobenius-fixed solutions

In [8]:
def normalized_frobenius(x):
    f = x.parent().frobenius_endomorphism()
    y = f(x)
    P = y.polynomial()
    j = 0

    while P[j] == 0:
        j += 1

    e = P[j]
    return y/e

def normalized_frobenius_orbit(x):
    orbit = [x]
    conj = normalized_frobenius(x)
    while not conj in orbit:
        orbit.append(conj)
        conj = normalized_frobenius(conj)
    return orbit

def normalized_frobenius_tuple(x):
    a, c = x
    f = a.parent().frobenius_endomorphism()
    b = f(a)
    P = b.polynomial()
    j = 0

    while P[j] == 0:
        j += 1

    e = P[j]
    return (b/e, c*e)

def normalized_frobenius_tuple_orbit(x):
    orbit = [x]
    conj = normalized_frobenius_tuple(x)
    while not conj in orbit:
        orbit.append(conj)
        conj = normalized_frobenius(conj)
    return orbit

def list_frobenius(L):
    M = [normalized_frobenius_tuple(x) for x in L]
    M.sort()
    return M

def list_frobenius_orbit(L):
    orbit = [L]
    conj = list_frobenius(L)
    while not conj in orbit:
        orbit.append(conj)
        conj = list_frobenius(conj)
    return orbit

In [9]:
def first_nonzero_coord(x):
    y = x.polynomial()
    j = 0
    while y[j] == 0:
        j += 1
    return j

def find_decomposition_frob(t, elems_global, index, upto, d, L, k = 0, G = [], W = None):
    if k <= upto:
        if W == None:
            W = t.parent().span([])
        if t in W:
            L.append(G)
        else:
            j = 0
            while B != []:
                b = B[0]
                orbit = normalized_frobenius((b, b.parent().one()))
                elems_in_orbit = [a for a,c in orbit]
                for elem in elems_in_orbit:
                    elems_global[firs_nonzero_coord(elem)].append(elem)
                
            
            
            1
            for j in range(len(B)):
                B2 = B[j+1:]
                G2 = copy(G)+[B[j]]
                find_decomposition(t, elems_global, index, upto, d, L, k+1, G2, W+t.parent().span([d[B[j]]]))

def finished_exhaustive_search(T, L, M, V, d):
    n = len(T)
    S = copy(T)
    elems = []
    for j in range(n):
        elems += [[]]
    for a in L:
        index = first_nonzero_coord(a)
        elems[index].append(a)
    for j in range(n):
        m = elems[j]
        try:
            W = V.subspace_with_basis([d[x] for x in m])
        except ValueError:
            return False
            
        if S[j] in W:
            coord = W.coordinates(S[j])
            for i in range(len(coord)):
                M.append((m[i], coord[i]))
                P = m[i].polynomial()
                for l in range(j, P.degree()+1):
                    S[l] = S[l] - P[l]*coord[i]*d[m[i]]
        else:
            return False

        
    return True
    
def tri_symmetric_frob_exhaustive_search(T, k, V, d, upto, Lglob, count = 0, orbits = None, chosen_orbits = []):
    
    if count <= upto:
        M = []
        
        if finished_exhaustive_search(T, chosen_orbits, M, V, d):

            M.sort()
            Lglob.append(M)

        elif count < upto:        
    
            if orbits == None:
                orbits = normalized_orbits(k)

            for j in range(len(orbits)):
                new_chosen_orbits = chosen_orbits+orbits[j]
                remaining_orbits = copy(orbits[j+1:])
                tri_symmetric_frob_exhaustive_search(T, k, V, d, upto, Lglob, count+len(orbits[j]), remaining_orbits, new_chosen_orbits)      

In [10]:
def normalized_orbits(k):
    n = k.degree()
    elems = [fields_elements(k, j) for j in range(n)]
    orbits = []
    j = 0
    while j < n:
        while elems[j] != []:
            x = elems[j][0]
            orb = normalized_frobenius_orbit(x)
            orbits.append(orb)
            for a in orb:
                index = first_nonzero_coord(a)
                elems[index].remove(a)
        j += 1
    return orbits

### Functions using matrix representation

In [11]:
def matrix_translation(I, J, S):
    n, m = S.dims()
    M = S()
    if I in ZZ and J in ZZ:
        M[I, J] = 1
        return M
    else:
        M = S()
        for i in range(n):
            a = I[i]
            if a == 0:
                continue
            for j in range(m):
                b = J[j]
                if b == 0:
                    continue
                else:
                    M[i, j] = a*b
        return M
    
"""
Compute the bilinear map associated to `x`, represented by an element in `S`.
""" 
def trace_to_bil_mat(x, S):
    n = S.ncols()
    g = x.parent().gen()
    L = [(x * g^j).trace() for j in range(n)]
    return matrix_translation(L, L, S)

"""
Compute a dictionnary associating elements of `k` with their bilinear form in `S`.
""" 
def make_dict_global_mat(k, S):
    d = dict()
    for x in k:
        d[x] = trace_to_bil_mat(x, S)
    return d

In [12]:
"""
Compute the list of vector of `V` associated with the multiplication bilinear form in `k`.
""" 
def multiplication_formula_mat(k, S):
    P = k.modulus()
    n = k.degree()
    T = (2*n-1)*[S()]
    for j in range(n):
        for i in range(n):
            T[i+j] += matrix_translation(j, i, S)
    
    l = len(T)
    
    while l > n:
        for j in range(P.degree()+1):
            T[l-1-n+j] -= P[j]*T[l-1]
        l -= 1
        T.pop(-1)
    
    return T

In [13]:
"""
Compute a list `L` of lists of elements that form a minimal decomposition 
of the matrix `t`.
"""
def find_decomposition_mat(t, B, d, L, G = []):
    if t.rank() == 0:
        L.append(G)
    else:
        r = t.rank()
        Field = t.base_ring()
        #nonZeroElems = [x for x in Field if x != 0]
        nonZeroElems = list(Field)[1:]
        elems = []
        for b in B:
            for x in nonZeroElems:
                if (t - x*d[b]).rank() < r:
                    elems.append((b, x))
                    break
        for j in range(len(elems)):
            y = elems[j]
            B2 = [elems[i][0] for i in range(j+1, len(elems))]
            find_decomposition_mat(t-y[1]*d[y[0]], B2, d, L, copy(G+[y]))

"""
Compute a list `Lglob` of tri-symmetric decompositions, where for each coordinate
the search is minimal.  
"""
def tri_symmetric_search_mat(T, k, d, Lglob, bound = Infinity, j = 0, Lloc = []):
    
    if finished(T):
        Lloc.sort()
        Lglob.append(Lloc)
    elif T[j].rank() <= bound:
        M = []
        find_decomposition_mat(T[j], fields_elements(k, j), d, M)

        for m in M:
            T2 = copy(T)
            T2[j] = 0
            Lloc2 = Lloc+m
            for x in m:
                P = x[0].polynomial()
                for l in range(j+1, P.degree()+1):
                    T2[l] = T2[l] - P[l]*x[1]*d[x[0]]
            
            tri_symmetric_search_mat(T2, k, d, Lglob, bound-len(m), j+1, Lloc2)

## Some other tests
Where we perform less rank tests by previously limitating the set of candidates. Unfortunately the time used to eliminate the candidates turns out to be non negligible and this strategy is not efficient.

In [14]:
"""
Compute a one-line matrix correspondind to the trace form of `field`
over its base field.
"""
def trace_form(field):
    base_field = field.base_ring()
    deg = field.degree()
    matrixSpace = MatrixSpace(base_field, 1, deg)
    gen = field.gen()
    trace = matrixSpace([(gen^j).trace() for j in range(deg)])
    return trace

"""
Compute a the multiplication-by-`element` matrix.
"""
def multiplication_matrix(element):
    field = element.parent()
    return element.polynomial()(companion_matrix(field.modulus()))

"""
Compute a the vector X such that X × transpose(X) is the matrix corresponding
to the map z |-> trace(`element` × z).
"""
def corresponding_vector(element):
    field = element.parent()
    vector_space = (field.base_ring())^field.degree()
    vector = vector_space((trace_form(field)*multiplication_matrix(element)).list())
    return vector

"""
Compute a dictionnary of elements in `field` and their corresponding vectors.
"""
def make_vectors(field):
    vectors = dict()
    for a in field:
        vectors[a] = corresponding_vector(a)
    return vectors

"""
Compute a list `L` of lists of elements that form a minimal decomposition 
of the matrix `t`.

# Notes:
    Needs the dictionnary `vectors`.
"""
def find_decomposition_mat2(t, B, d, vectors, L, G = []):
    global CPT, CPT2
    if t.rank() == 0:
        L.append(G)
    else:
        r = t.rank()
        Field = t.base_ring()
        nonZeroElems = [x for x in Field if x != 0]
        elems = []
        Img = t.image()
        for b in B:
            if vectors[b] in Img:
                CPT2 += 1
                for x in nonZeroElems:
                    if (t - x*d[b]).rank() < r:
                        CPT += 1
                        elems.append((b, x))
                        break
        for j in range(len(elems)):
            y = elems[j]
            B2 = [elems[i][0] for i in range(j+1, len(elems))]
            find_decomposition_mat2(t-y[1]*d[y[0]], B2, d, vectors, L, copy(G+[y]))
            
"""
Compute a list `Lglob` of tri-symmetric decompositions, where for each coordinate
the search is minimal.

# Notes:
    Needs the dictionnary `vectors`.  
"""
def tri_symmetric_search_mat2(T, k, d, vectors, Lglob, bound = Infinity, j = 0, Lloc = []):
    if finished(T):
        Lloc.sort()
        Lglob.append(Lloc)
    elif T[j].rank() <= bound:
        M = []
        find_decomposition_mat2(T[j], fields_elements(k, j), d, vectors, M)

        for m in M:
            T2 = copy(T)
            T2[j] = 0
            Lloc2 = Lloc+m
            for x in m:
                P = x[0].polynomial()
                for l in range(j+1, P.degree()+1):
                    T2[l] = T2[l] - P[l]*x[1]*d[x[0]]
            
            tri_symmetric_search_mat2(T2, k, d, vectors, Lglob, bound-len(m), j+1, Lloc2)

# $2\times3$ terms polynomial product

In [15]:
T = [b(1, 2), b(1, 1) + b(0, 2), b(0, 1) + b(1, 0), b(0, 0)]

NameError: name 'b' is not defined

In [15]:
L = []
expand_subspace(T, 2, 3, L=L);

NameError: name 'T' is not defined

In [80]:
W = L[0]

In [81]:
G = rank_one_elements(V, 2, 3)
B = rank_one_basis(W, G)

In [82]:
B

[(1, 0, 0, 0, 0, 0),
 (0, 0, 0, 1, 0, 0),
 (0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 0, 1),
 (1, 1, 1, 1, 0, 0)]

In [86]:
t = T[0]

In [87]:
U = V.span(T)

In [88]:
U.dimension()

4

In [89]:
WW = V.span_of_basis(B)

In [91]:
formulas(T, W, G)

([(1, 0, 0, 0, 0, 0),
  (0, 0, 0, 1, 0, 0),
  (0, 0, 0, 0, 1, 0),
  (0, 0, 0, 0, 0, 1),
  (1, 1, 1, 1, 0, 0)],
 [((0, 0, 0, 0, 0, 1), [0, 0, 0, 1, 0]),
  ((0, 0, 0, 1, 1, 0), [0, 1, 1, 0, 0]),
  ((0, 1, 1, 0, 0, 0), [1, 1, 0, 0, 1]),
  ((1, 0, 0, 0, 0, 0), [1, 0, 0, 0, 0])])

In [92]:
V, n, m = GF(2)^4, 2, 2

In [93]:
T = [b(1, 1, V, n, m), b(0, 1, V, n, m)+b(1, 0, V, n, m) ,b(0, 0, V, n, m)]

In [94]:
L = []
expand_subspace(T, n, m, L=L)

In [95]:
W = L[0]

In [96]:
G = rank_one_elements(V, n, m)

In [97]:
formulas(T, W, G)

([(1, 0, 0, 0), (0, 0, 0, 1), (1, 1, 1, 1)],
 [((0, 0, 0, 1), [0, 1, 0]),
  ((0, 1, 1, 0), [1, 1, 1]),
  ((1, 0, 0, 0), [1, 0, 0])])

In [58]:
W

Vector space of degree 4 and dimension 2 over Finite Field of size 2
Basis matrix:
[1 0 0 0]
[0 0 1 0]

In [61]:
V.span([x for x in G if x in V.span(T)])

Vector space of degree 4 and dimension 2 over Finite Field of size 2
Basis matrix:
[1 0 0 0]
[0 0 1 0]

In [62]:
V.span(T)

Vector space of degree 4 and dimension 2 over Finite Field of size 2
Basis matrix:
[1 0 0 0]
[0 0 1 0]

In [65]:
T

[(0, 0, 1, 0), (0, 0, 0, 0), (1, 0, 0, 0)]

In [67]:
b(0, 1, V, n, m)

(0, 1, 0, 0)

In [68]:
b(1, 0, V, n, m)

(0, 1, 0, 0)

# $2\times2$ symmetric matrices over $\mathbb F_2$

In [11]:
V, n, m = VectorSpace(GF(2), 9), 3, 3

In [12]:
T = [b(0, 0, V, n, m)+b(1, 1, V, n, m), b(0, 1, V, n, m)+b(1, 2, V, n, m), b(1, 0, V, n, m)+b(2, 1, V, n, m), b(1, 1, V, n, m)+b(2, 2, V, n, m)]

In [77]:
T

[(1, 0, 0, 0, 1, 0, 0, 0, 0),
 (0, 0, 0, 1, 0, 0, 0, 1, 0),
 (0, 1, 0, 0, 0, 1, 0, 0, 0),
 (0, 0, 0, 0, 1, 0, 0, 0, 1)]

In [14]:
V.span(T)

Vector space of degree 9 and dimension 4 over Finite Field of size 2
Basis matrix:
[1 0 0 0 0 0 0 0 1]
[0 1 0 0 0 1 0 0 0]
[0 0 0 1 0 0 0 1 0]
[0 0 0 0 1 0 0 0 1]

In [62]:
L = []
%time expand_subspace(T, n, m, upto = 5, L=L)

CPU times: user 418 ms, sys: 28.8 ms, total: 447 ms
Wall time: 407 ms


In [81]:
len(L)

0

In [67]:
W = L[0]

In [29]:
cpt = 0

for l in L:
    if l.dimension() == 6:
        cpt += 1
        U = l
cpt
U

Vector space of degree 9 and dimension 6 over Finite Field of size 2
Basis matrix:
[1 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 1 0]
[0 0 0 1 0 0 0 1 0]
[0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 1 0 1 0]
[0 0 0 0 0 0 0 0 1]

In [24]:
dims = [l.dimension() for l in L]

In [25]:
min(dims)

6

In [60]:
U

Vector space of degree 9 and dimension 6 over Finite Field of size 2
Basis matrix:
[1 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 1 0]
[0 0 0 1 0 0 0 1 0]
[0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 1 0 1 0]
[0 0 0 0 0 0 0 0 1]

In [32]:
G = rank_one_elements(V, n, m)

In [33]:
len(G)

49

In [62]:
B = rank_one_basis(U, G)

In [63]:
B

[(1, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 1, 0, 0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 1, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 1),
 (0, 0, 0, 1, 0, 1, 0, 0, 0),
 (1, 1, 0, 1, 1, 0, 0, 0, 0)]

In [71]:
f = formulas(T, W, G)

In [72]:
f[0]

[(1, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 1, 0, 0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 1, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 1),
 (0, 0, 0, 1, 0, 1, 0, 0, 0),
 (1, 1, 0, 1, 1, 0, 0, 0, 0)]

In [73]:
f[1]

[((1, 0, 0, 0, 1, 0, 0, 0, 0), [1, 0, 1, 0, 0, 0]),
 ((0, 0, 0, 1, 0, 0, 0, 1, 0), [1, 1, 1, 0, 0, 1]),
 ((0, 1, 0, 0, 0, 1, 0, 0, 0), [1, 0, 1, 0, 1, 1]),
 ((0, 0, 0, 0, 1, 0, 0, 0, 1), [0, 0, 1, 1, 0, 0])]

# $2\times2$ matrices over $\mathbb F_2$

In [31]:
V, n, m = VectorSpace(GF(2), 16), 4, 4

In [32]:
%time G = rank_one_elements(V, n, m)

CPU times: user 34.4 ms, sys: 4.03 ms, total: 38.4 ms
Wall time: 30.7 ms


In [10]:
len(G)

225

In [ ]:
# T = [b(0, 0, V, n, m)+b(1, 2, V, n, m), b(0, 1, V, n, m)+b(1, 3, V, n, m), b(2, 0, V, n, m)+b(3, 2, V, n, m), b(2, 1, V, n, m)+b(3, 3, V, n, m)]

In [15]:
T

[(1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0),
 (0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1)]

In [17]:
L = []
%time expand_subspace(T, n, m, upto = 5, L=L)

CPU times: user 1min 46s, sys: 184 ms, total: 1min 47s
Wall time: 1min 46s


In [24]:
L = []
%time expand_subspace2(T, n, m, upto = 5, L=L)

CPU times: user 2min, sys: 333 ms, total: 2min
Wall time: 2min


In [25]:
L

[]

In [26]:
L2 = [g for g in G if g not in V.span(T)]

In [27]:
len(L2)

225

In [34]:
W = V.span(T)

In [35]:
W.intersection(V.span(G))

Vector space of degree 16 and dimension 4 over Finite Field of size 2
Basis matrix:
[1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1]

# Multiplication in $\mathbb F_4/\mathbb F_2$

In [3]:
k = GF(4)
k.modulus()

x^2 + x + 1

In [4]:
n, m = 2, 2
V = VectorSpace(GF(2), n*m)

In [66]:
T = [b(0, 0, V, n, m)+b(1, 1, V, n, m), b(0, 1, V, n, m)+b(1, 0, V, n, m)+ b(1, 1, V, n, m)]

In [67]:
G = rank_one_elements(V, n, m)

In [73]:
L = []
%time expand_subspace(T, n, m, upto = 3, L=L)

CPU times: user 211 ms, sys: 20.6 ms, total: 232 ms
Wall time: 209 ms


In [74]:
L

[Vector space of degree 4 and dimension 3 over Finite Field of size 2
 Basis matrix:
 [1 0 0 0]
 [0 1 1 0]
 [0 0 0 1],
 Vector space of degree 4 and dimension 3 over Finite Field of size 2
 Basis matrix:
 [1 0 0 1]
 [0 1 0 0]
 [0 0 1 1],
 Vector space of degree 4 and dimension 3 over Finite Field of size 2
 Basis matrix:
 [1 0 0 1]
 [0 1 0 1]
 [0 0 1 0],
 Vector space of degree 4 and dimension 3 over Finite Field of size 2
 Basis matrix:
 [1 0 0 1]
 [0 1 0 1]
 [0 0 1 0],
 Vector space of degree 4 and dimension 3 over Finite Field of size 2
 Basis matrix:
 [1 0 0 0]
 [0 1 1 0]
 [0 0 0 1],
 Vector space of degree 4 and dimension 3 over Finite Field of size 2
 Basis matrix:
 [1 0 0 1]
 [0 1 0 0]
 [0 0 1 1],
 Vector space of degree 4 and dimension 3 over Finite Field of size 2
 Basis matrix:
 [1 0 0 1]
 [0 1 0 0]
 [0 0 1 1],
 Vector space of degree 4 and dimension 3 over Finite Field of size 2
 Basis matrix:
 [1 0 0 1]
 [0 1 0 1]
 [0 0 1 0],
 Vector space of degree 4 and dimension 3 over F

In [78]:
for l in L:
    print rank_one_basis(l, G)

[(1, 0, 0, 0), (0, 0, 0, 1), (1, 1, 1, 1)]
[(0, 1, 0, 0), (0, 0, 1, 1), (1, 0, 1, 0)]
[(0, 1, 0, 1), (0, 0, 1, 0), (1, 1, 0, 0)]
[(0, 1, 0, 1), (0, 0, 1, 0), (1, 1, 0, 0)]
[(1, 0, 0, 0), (0, 0, 0, 1), (1, 1, 1, 1)]
[(0, 1, 0, 0), (0, 0, 1, 1), (1, 0, 1, 0)]
[(0, 1, 0, 0), (0, 0, 1, 1), (1, 0, 1, 0)]
[(0, 1, 0, 1), (0, 0, 1, 0), (1, 1, 0, 0)]
[(1, 0, 0, 0), (0, 0, 0, 1), (1, 1, 1, 1)]


In [45]:
w = [b(0, 0, V, n, m), b(1, 1, V, n, m), b([1, 1], [1, 1], V, n, m)]

In [46]:
w

[(1, 0, 0, 0), (0, 0, 0, 1), (1, 1, 1, 1)]

In [47]:
W = V.span(w)

In [49]:
rank_one_basis(W, G)

[(1, 0, 0, 0), (0, 0, 0, 1), (1, 1, 1, 1)]

In [50]:
W

Vector space of degree 4 and dimension 3 over Finite Field of size 2
Basis matrix:
[1 0 0 0]
[0 1 1 0]
[0 0 0 1]

In [51]:
V.span(T).dimension()

2

In [55]:
V.span([g for g in G if g in V.span(T)+V.span([b([1, 1], [1, 1], V, n, m)])]).dimension()

3

In [69]:
V.span(T).dimension()

2

In [70]:
G

[(1, 0, 0, 0),
 (1, 0, 1, 0),
 (0, 0, 1, 0),
 (1, 1, 0, 0),
 (1, 1, 1, 1),
 (0, 0, 1, 1),
 (0, 1, 0, 0),
 (0, 1, 0, 1),
 (0, 0, 0, 1)]

# Multiplication in $\mathbb F_8/\mathbb F_2$

In [10]:
k = GF(8)

In [11]:
k.modulus()

x^3 + x + 1

In [12]:
n, m = 3, 3
V = VectorSpace(GF(2), n*m)

In [120]:
T = [b(0, 0, V, n, m)+b(2, 1, V, n, m)+b(1, 2, V, n, m), b(2, 2, V, n, m)+b(2, 1, V, n, m)+b(1, 2, V, n, m)+b(1, 0, V, n, m)+b(0, 1, V, n, m), b(2, 0, V, n, m)+b(1, 1, V, n, m)+b(0, 2, V, n, m)+b(2, 2, V, n, m)]

In [156]:
L = []
%time expand_subspace2(T, n, m, upto = 6, L=L)

CPU times: user 2min 14s, sys: 197 ms, total: 2min 14s
Wall time: 2min 14s


# Tri-symmetric study of $\mathbb F_4/\mathbb F_2$

In [19]:
k = GF(4)

In [20]:
n, m = 2, 2
V = VectorSpace(GF(2), n*m)

In [21]:
k.modulus()

x^2 + x + 1

In [31]:
T = [b(0, 0, V, n, m)+b(1, 1, V, n, m), b(0, 1, V, n, m)+b(1, 0, V, n, m)+ b(1, 1, V, n, m)]

In [22]:
x = k.gen()

In [24]:
k(x).trace()

1

In [25]:
x.trace()

0

In [26]:
(x^2).trace()

0

In [30]:
trace_to_bil(k(1), V)

(0, 0, 0, 1)

In [35]:
for j in range(len(T)):
    print T[j] in V.span([trace_to_bil(y, V) for y in k if y != 0])

True
True


# Tri-symmetric study of $\mathbb F_8/\mathbb F_2$

In [48]:
k = GF(8)
x = k.gen()

In [41]:
n, m = 3, 3
V = VectorSpace(GF(2), n*m)

In [42]:
T = [b(0, 0, V, n, m)+b(2, 1, V, n, m)+b(1, 2, V, n, m), b(2, 2, V, n, m)+b(2, 1, V, n, m)+b(1, 2, V, n, m)+b(1, 0, V, n, m)+b(0, 1, V, n, m), b(2, 0, V, n, m)+b(1, 1, V, n, m)+b(0, 2, V, n, m)+b(2, 2, V, n, m)]

In [43]:
W = V.span([trace_to_bil(y, V) for y in k if y != 0])

In [44]:
W.dimension()

6

In [45]:
for j in range(len(T)):
    print T[j] in W

True
True
True


In [46]:
W

Vector space of degree 9 and dimension 6 over Finite Field of size 2
Basis matrix:
[1 0 0 0 0 0 0 0 0]
[0 1 0 1 0 0 0 0 0]
[0 0 1 0 0 0 1 0 0]
[0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 1 0 1 0]
[0 0 0 0 0 0 0 0 1]

In [52]:
trace_to_bil(k(1+x), V)

(1, 0, 1, 0, 0, 0, 1, 0, 1)

In [53]:
T[0]

(1, 0, 0, 0, 0, 1, 0, 1, 0)

In [55]:
T

[(1, 0, 0, 0, 0, 1, 0, 1, 0),
 (0, 1, 0, 1, 0, 1, 0, 1, 1),
 (0, 0, 1, 0, 1, 0, 1, 0, 1)]

### First coordinate `T[0]`

In [56]:
trace_to_bil(k(1), V)

(1, 0, 0, 0, 0, 0, 0, 0, 0)

In [57]:
trace_to_bil(k(1+x), V)

(1, 0, 1, 0, 0, 0, 1, 0, 1)

In [58]:
trace_to_bil(k(1)+x+x^2, V)

(1, 1, 1, 1, 1, 1, 1, 1, 1)

In [59]:
trace_to_bil(k(1)+x^2, V)

(1, 1, 0, 1, 1, 0, 0, 0, 0)

In [65]:
trace_to_bil(k(1)+x+x^2, V)+T[0]

(0, 1, 1, 1, 1, 0, 1, 0, 1)

In [64]:
trace_to_bil(k(1+x), V)+trace_to_bil(k(1)+x^2, V)+trace_to_bil(k(1)+x+x^2, V)+T[0]

(0, 0, 0, 0, 0, 0, 0, 0, 0)

### Second coordinate `T[1]`

In [66]:
trace_to_bil(k(1+x), V)+trace_to_bil(k(1)+x+x^2, V)+T[1]

(0, 0, 0, 0, 1, 0, 0, 0, 1)

In [69]:
trace_to_bil(x, V)

(0, 0, 0, 0, 0, 0, 0, 0, 1)

In [68]:
trace_to_bil(x+x^2, V)

(0, 0, 0, 0, 1, 1, 0, 1, 1)

### Third coordinate `T[2]`

In [71]:
T[2]+trace_to_bil(k(1)+x^2, V)+trace_to_bil(k(1)+x+x^2, V)

(0, 0, 0, 0, 1, 1, 0, 1, 0)

In [72]:
trace_to_bil(x+x^2, V)

(0, 0, 0, 0, 1, 1, 0, 1, 1)

In [73]:
trace_to_bil(x^2, V)

(0, 0, 0, 0, 1, 0, 0, 0, 0)

# Tri-symmetric study of $\mathbb F_9/\mathbb F_3$

In [78]:
k = GF(9)
x = gen(k)
k.modulus()

x^2 + 2*x + 2

In [85]:
k.gen()^3

2*z2 + 1

In [79]:
n, m = 2, 2
V = VectorSpace(GF(3), n*m)
d = make_dict_global(k, V)

In [80]:
T = [b(0, 0, V, n, m)+b(1, 1, V, n, m), b(0, 1, V, n, m)+b(1, 0, V, n, m)+ b(1, 1, V, n, m)]
T

[(1, 0, 0, 1), (0, 1, 1, 1)]

In [81]:
T2 = multiplication_formula(k, V)

In [82]:
T2

[(1, 0, 0, 1), (0, 1, 1, 1)]

In [83]:
T == T2

True

In [84]:
L = []
tri_symmetric_search(T, 0, k, V, d, 3, 3, L)
L

[[(2*z2 + 1, 2), (1, 2), (z2, 2)]]

# Tri-symmetric study of $\mathbb F_{27}/\mathbb F_3$

In [50]:
k = GF(27)
x = gen(k)
k.modulus()

x^3 + 2*x + 1

In [51]:
n, m = 3, 3
V = VectorSpace(GF(3), n*m)
d = make_dict_global(k, V)

In [52]:
T = multiplication_formula(k, V)
T

[(1, 0, 0, 0, 0, 2, 0, 2, 0),
 (0, 1, 0, 1, 0, 1, 0, 1, 2),
 (0, 0, 1, 0, 1, 0, 1, 0, 1)]

In [83]:
t = T[0]

In [32]:
B = fields_elements(k, 2)
B

[z3^2]

In [74]:
k.modulus()

x^3 + 2*x + 1

In [75]:
x = k.gen()

In [76]:
x^3

z3 + 2

In [77]:
x^9

z3 + 1

In [84]:
M = []
find_decomposition(t, fields_elements(k, 0), 3, d, M)
M

[[z3 + 1, 2*z3 + 1, 2*z3^2 + 1], [z3^2 + 2*z3 + 1, 2*z3^2 + z3 + 1, z3^2 + 1]]

In [60]:
M = []
%time tri_symmetric_search(T, 0, k, V, d, 6, 6, M)
M

CPU times: user 1.65 s, sys: 32.3 ms, total: 1.68 s
Wall time: 1.66 s


[[(z3^2 + z3, 1),
  (z3^2 + z3 + 1, 2),
  (z3^2 + 2*z3 + 1, 1),
  (2*z3^2 + 1, 1),
  (2*z3^2 + z3 + 1, 2),
  (2*z3^2 + 2*z3 + 1, 1)],
 [(z3^2, 2),
  (z3^2 + 1, 2),
  (z3^2 + z3 + 1, 1),
  (2*z3^2 + 1, 2),
  (2*z3^2 + z3, 1),
  (2*z3^2 + 2*z3 + 1, 2)],
 [(z3, 1),
  (z3 + 1, 1),
  (2*z3 + 1, 2),
  (z3^2 + z3, 2),
  (2*z3^2 + 1, 1),
  (2*z3^2 + z3, 1)],
 [(z3^2, 1),
  (z3^2 + 1, 1),
  (z3^2 + z3, 2),
  (z3^2 + 2*z3 + 1, 2),
  (2*z3^2 + z3, 2),
  (2*z3^2 + z3 + 1, 1)]]

# Tri-symmetric study of $\mathbb F_{3^4}/\mathbb F_3$

In [218]:
k = GF(3^4)
x = gen(k)
k.modulus()

x^4 + 2*x^3 + 2

In [219]:
n, m = 4, 4
V = VectorSpace(GF(3), n*m)
d = make_dict_global(k, V)

In [220]:
T = multiplication_formula(k, V)

In [66]:
T

[(1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1),
 (0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1),
 (0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1),
 (0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1)]

In [221]:
L = []
%time tri_symmetric_search(T, 0, k, V, d, 9, 4, L)
len(L)

CPU times: user 2min 42s, sys: 1.15 s, total: 2min 43s
Wall time: 2min 44s


5

In [222]:
L[0]

[(z4, 2),
 (z4^2, 2),
 (z4^3, 2),
 (z4^3 + 1, 2),
 (z4^3 + z4, 2),
 (z4^3 + z4^2, 1),
 (z4^3 + 2*z4^2 + 1, 2),
 (2*z4^3 + z4^2 + z4 + 1, 1),
 (2*z4^3 + 2*z4^2 + 2*z4 + 1, 2)]

In [245]:
M = []
%time find_decomposition(T[0], fields_elements(k, 0), 3, d, M)
M

CPU times: user 33.2 s, sys: 227 ms, total: 33.5 s
Wall time: 33.5 s


[]

# Tri-symmetric study of $\mathbb F_{3^5}/\mathbb F_3$

In [9]:
n = 5
k = GF(3^n)
x = gen(k)
V = VectorSpace(GF(3), n^2)
d = make_dict_global(k, V)

In [10]:
T = multiplication_formula(k, V)

In [11]:
T

[(1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0),
 (0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0),
 (0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 2, 0, 0, 1, 2, 0),
 (0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 2),
 (0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1)]

In [12]:
L = []
%time tri_symmetric_search(T, 0, k, V, d, 11, 5, L)
L

KeyboardInterrupt: 

# Playing with automorphisms

In [110]:
k = GF(27)
x = gen(k)
k.modulus()

x^3 + 2*x + 1

In [111]:
n, m = 3, 3
V = VectorSpace(GF(3), n*m)
d = make_dict_global(k, V)

In [14]:
T = multiplication_formula(k, V)

In [15]:
L = []
%time tri_symmetric_search(T, 0, k, V, d, 6, 6, L)
L

CPU times: user 1.66 s, sys: 24.1 ms, total: 1.68 s
Wall time: 1.67 s


[[(z3^2 + z3, 1),
  (z3^2 + z3 + 1, 2),
  (z3^2 + 2*z3 + 1, 1),
  (2*z3^2 + 1, 1),
  (2*z3^2 + z3 + 1, 2),
  (2*z3^2 + 2*z3 + 1, 1)],
 [(z3^2, 2),
  (z3^2 + 1, 2),
  (z3^2 + z3 + 1, 1),
  (2*z3^2 + 1, 2),
  (2*z3^2 + z3, 1),
  (2*z3^2 + 2*z3 + 1, 2)],
 [(z3, 1),
  (z3 + 1, 1),
  (2*z3 + 1, 2),
  (z3^2 + z3, 2),
  (2*z3^2 + 1, 1),
  (2*z3^2 + z3, 1)],
 [(z3^2, 1),
  (z3^2 + 1, 1),
  (z3^2 + z3, 2),
  (z3^2 + 2*z3 + 1, 2),
  (2*z3^2 + z3, 2),
  (2*z3^2 + z3 + 1, 1)]]

In [20]:
M =[]
%time tri_symmetric_frob_exhaustive_search(T, k, V, d, 6, M)
M

CPU times: user 65.2 ms, sys: 19.9 ms, total: 85.1 ms
Wall time: 75.8 ms


[[(z3, 1),
  (z3 + 1, 1),
  (2*z3 + 1, 2),
  (z3^2 + z3, 2),
  (2*z3^2 + 1, 1),
  (2*z3^2 + z3, 1)]]

# $\mathbb F_{3^4}$

In [8]:
p, n = 3, 4
k = GF(p^n)
V = VectorSpace(GF(p), n^2)
d = make_dict_global(k, V)
T = multiplication_formula(k, V)

In [27]:
L = []
%time tri_symmetric_search(T, 0, k, V, d, 9, 4, L)
#L

CPU times: user 2min 54s, sys: 1.28 s, total: 2min 55s
Wall time: 2min 55s


In [26]:
M =[]
%time tri_symmetric_frob_exhaustive_search(T, k, V, d, 9, M)
M

CPU times: user 790 ms, sys: 36 ms, total: 826 ms
Wall time: 812 ms


[[(1, 2),
  (z4, 2),
  (z4^2 + z4 + 1, 1),
  (2*z4^2 + z4, 2),
  (z4^3, 2),
  (z4^3 + 2*z4^2 + 1, 2),
  (z4^3 + 2*z4^2 + 2*z4 + 1, 2),
  (2*z4^3 + z4, 2),
  (2*z4^3 + 2*z4^2 + z4, 1)]]

In [26]:
M

[[(1, 1),
  (2*z5 + 1, 2),
  (2*z5^2 + 2*z5 + 1, 1),
  (z5^3 + 2*z5 + 1, 1),
  (2*z5^3 + 1, 2),
  (2*z5^3 + 2*z5^2 + z5 + 1, 1),
  (2*z5^4 + z5 + 1, 1),
  (2*z5^4 + 2*z5^2 + 1, 1),
  (2*z5^4 + 2*z5^3 + z5, 1),
  (2*z5^4 + 2*z5^3 + z5^2 + 1, 2),
  (2*z5^4 + 2*z5^3 + z5^2 + z5 + 1, 1)]]

In [28]:
for l in L:
    print len(list_frobenius_orbit(l)) == 1

False
False
False
False
True


In [30]:
L[-1] == M[0]

True

# $\mathbb F_{3^5}$

In [156]:
p, n = 3, 5
k = GF(p^n)
V = VectorSpace(GF(p), n^2)
d = make_dict_global(k, V)
T = multiplication_formula(k, V)

In [157]:
M =[]
%time tri_symmetric_frob_exhaustive_search(T, k, V, d, 11, M)
M

CPU times: user 1.53 s, sys: 12.2 ms, total: 1.54 s
Wall time: 1.52 s


[[(1, 1),
  (2*z5 + 1, 2),
  (2*z5^2 + 2*z5 + 1, 1),
  (z5^3 + 2*z5 + 1, 1),
  (2*z5^3 + 1, 2),
  (2*z5^3 + 2*z5^2 + z5 + 1, 1),
  (2*z5^4 + z5 + 1, 1),
  (2*z5^4 + 2*z5^2 + 1, 1),
  (2*z5^4 + 2*z5^3 + z5, 1),
  (2*z5^4 + 2*z5^3 + z5^2 + 1, 2),
  (2*z5^4 + 2*z5^3 + z5^2 + z5 + 1, 1)]]

In [18]:
for x in M[0]:
    print (x[1]*x[0]).multiplicative_order()

1
242
11
11
242
11
242
242
11
11
242


In [159]:
len(M[0])

11

# $\mathbb F_{3^6}$

In [42]:
p, n = 3, 6
k = GF(p^n)
V = VectorSpace(GF(p), n^2)
d = make_dict_global(k, V)
T = multiplication_formula(k, V)

In [40]:
M =[]
%time tri_symmetric_frob_exhaustive_search(T, k, V, d, 15, M)
M

CPU times: user 2min 14s, sys: 472 ms, total: 2min 14s
Wall time: 2min 14s


[[(2*z6^4 + 1, 1),
  (2*z6^4 + 2*z6^2 + z6 + 1, 1),
  (2*z6^4 + z6^3 + 1, 2),
  (2*z6^4 + z6^3 + z6 + 1, 1),
  (z6^5 + z6^3 + 1, 2),
  (z6^5 + z6^3 + 2*z6^2 + 2*z6 + 1, 1),
  (z6^5 + 2*z6^3 + 1, 1),
  (z6^5 + z6^4 + z6^3 + 2*z6 + 1, 1),
  (z6^5 + z6^4 + 2*z6^3 + z6, 1),
  (z6^5 + z6^4 + 2*z6^3 + 2*z6^2 + 2*z6 + 1, 2),
  (z6^5 + 2*z6^4 + z6^3 + 2*z6^2 + 2*z6 + 1, 1),
  (z6^5 + 2*z6^4 + 2*z6^3 + 1, 1),
  (z6^5 + 2*z6^4 + 2*z6^3 + z6 + 1, 2),
  (2*z6^5 + z6^2 + 2*z6 + 1, 1),
  (2*z6^5 + z6^4 + z6^3 + 2*z6^2 + 2*z6 + 1, 2)],
 [(2*z6^4 + z6^2 + z6 + 1, 2),
  (z6^5 + z6 + 1, 1),
  (z6^5 + z6^3 + z6^2 + z6 + 1, 2),
  (z6^5 + z6^4, 2),
  (z6^5 + z6^4 + 1, 2),
  (z6^5 + 2*z6^4 + z6, 2),
  (2*z6^5 + z6^3 + z6 + 1, 2),
  (2*z6^5 + z6^3 + 2*z6 + 1, 2),
  (2*z6^5 + z6^3 + 2*z6^2 + 1, 1),
  (2*z6^5 + z6^3 + 2*z6^2 + z6, 1),
  (2*z6^5 + 2*z6^4 + 1, 1),
  (2*z6^5 + 2*z6^4 + z6^2 + z6, 1),
  (2*z6^5 + 2*z6^4 + z6^3 + z6^2, 1),
  (2*z6^5 + 2*z6^4 + 2*z6^3 + z6^2 + z6, 1),
  (2*z6^5 + 2*z6^4 + 2*z6^3 + 2

In [41]:
len(M[0]), len(M[1])

(15, 15)

# $\mathbb F_{3^7}$

In [15]:
p, n = 3, 7
k = GF(p^n)
V = VectorSpace(GF(p), n^2)
d = make_dict_global(k, V)
T = multiplication_formula(k, V)

In [16]:
M =[]
%time tri_symmetric_frob_exhaustive_search(T, k, V, d, 20, M)
M

CPU times: user 1min 23s, sys: 199 ms, total: 1min 23s
Wall time: 1min 23s


[]

In [18]:
len(normalized_orbits(GF(3^8)))

423

# $\mathbb F_{3^8}$

In [20]:
p, n = 3, 8
k = GF(p^n)
V = VectorSpace(GF(p), n^2)
d = make_dict_global(k, V)
T = multiplication_formula(k, V)

In [23]:
M = []
%time tri_symmetric_frob_exhaustive_search(T, k, V, d, 15, M)
M

CPU times: user 7min 42s, sys: 1.12 s, total: 7min 43s
Wall time: 7min 43s


[]

In [ ]:
# ancre

# Things with matrices

In [147]:
p, n = 3, 3
k = GF(p^n)
S = MatrixSpace(GF(p), n, n)
d = make_dict_global_mat(k, S)
vectors = make_vectors(k)
T = multiplication_formula_mat(k, S)

In [48]:
L = []
%time tri_symmetric_search_mat(T, k, d, L, bound = 6)
L

CPU times: user 12.1 ms, sys: 0 ns, total: 12.1 ms
Wall time: 11.1 ms


[[(z3^2, 1),
  (z3^2 + 1, 1),
  (z3^2 + z3, 2),
  (z3^2 + 2*z3 + 1, 2),
  (2*z3^2 + z3, 2),
  (2*z3^2 + z3 + 1, 1)]]

In [135]:
L3 = []
%time tri_symmetric_search_mat3(T, k, d, L3, counts = [2, 1, 0], bound = 6)
L3

CPU times: user 86.7 ms, sys: 20.2 ms, total: 107 ms
Wall time: 96 ms


[[(z3^2, 2),
  (z3^2 + 1, 2),
  (z3^2 + z3 + 1, 1),
  (2*z3^2 + 1, 2),
  (2*z3^2 + z3, 1),
  (2*z3^2 + 2*z3 + 1, 2)],
 [(z3, 1),
  (z3 + 1, 1),
  (2*z3 + 1, 2),
  (z3^2 + z3, 2),
  (2*z3^2 + 1, 1),
  (2*z3^2 + z3, 1)],
 [(z3^2, 1),
  (z3^2 + 1, 1),
  (z3^2 + z3, 2),
  (z3^2 + 2*z3 + 1, 2),
  (2*z3^2 + z3, 2),
  (2*z3^2 + z3 + 1, 1)]]

In [133]:
L3 = []
%time tri_symmetric_search_mat3(T, k, d, L3, counts = [2, 1, 0], bound = 6)
L3

CPU times: user 103 ms, sys: 8.61 ms, total: 112 ms
Wall time: 102 ms


[[(z3^2, 2),
  (z3^2 + 1, 2),
  (z3^2 + z3 + 1, 1),
  (2*z3^2 + 1, 2),
  (2*z3^2 + z3, 1),
  (2*z3^2 + 2*z3 + 1, 2)],
 [(z3^2 + z3, 1),
  (z3^2 + z3 + 1, 2),
  (z3^2 + 2*z3 + 1, 1),
  (2*z3^2 + 1, 1),
  (2*z3^2 + z3 + 1, 2),
  (2*z3^2 + 2*z3 + 1, 1)],
 [(z3, 1),
  (z3 + 1, 1),
  (2*z3 + 1, 2),
  (z3^2 + z3, 2),
  (2*z3^2 + 1, 1),
  (2*z3^2 + z3, 1)],
 [(z3^2, 1),
  (z3^2 + 1, 1),
  (z3^2 + z3, 2),
  (z3^2 + 2*z3 + 1, 2),
  (2*z3^2 + z3, 2),
  (2*z3^2 + z3 + 1, 1)]]

In [180]:
L, L3, L4, L4bis = test_algs(T, k, d, counts = [2, 1, 0], bound = 6)

*** Algo 1 ***
CPU times: user 4.93 ms, sys: 70 µs, total: 5 ms
Wall time: 4.62 ms
*** Algo 3 ***
CPU times: user 58.1 ms, sys: 0 ns, total: 58.1 ms
Wall time: 59.2 ms
*** Algo 4 ***
CPU times: user 27.6 ms, sys: 4.01 ms, total: 31.6 ms
Wall time: 28.6 ms
*** Algo 4bis ***
CPU times: user 29.6 ms, sys: 71 µs, total: 29.7 ms
Wall time: 26.7 ms


In [181]:
L

[[(z3^2, 1),
  (z3^2 + 1, 1),
  (z3^2 + z3, 2),
  (z3^2 + 2*z3 + 1, 2),
  (2*z3^2 + z3, 2),
  (2*z3^2 + z3 + 1, 1)]]

In [182]:
L3

[[(z3^2, 2),
  (z3^2 + 1, 2),
  (z3^2 + z3 + 1, 1),
  (2*z3^2 + 1, 2),
  (2*z3^2 + z3, 1),
  (2*z3^2 + 2*z3 + 1, 2)],
 [(z3, 1),
  (z3 + 1, 1),
  (2*z3 + 1, 2),
  (z3^2 + z3, 2),
  (2*z3^2 + 1, 1),
  (2*z3^2 + z3, 1)],
 [(z3^2, 1),
  (z3^2 + 1, 1),
  (z3^2 + z3, 2),
  (z3^2 + 2*z3 + 1, 2),
  (2*z3^2 + z3, 2),
  (2*z3^2 + z3 + 1, 1)]]

In [183]:
L4 == L4bis

True

In [184]:
L4

[[(z3^2, 2),
  (z3^2 + 1, 2),
  (z3^2 + z3 + 1, 1),
  (2*z3^2 + 1, 2),
  (2*z3^2 + z3, 1),
  (2*z3^2 + 2*z3 + 1, 2)],
 [(z3^2 + z3, 1),
  (z3^2 + z3 + 1, 2),
  (z3^2 + 2*z3 + 1, 1),
  (2*z3^2 + 1, 1),
  (2*z3^2 + z3 + 1, 2),
  (2*z3^2 + 2*z3 + 1, 1)],
 [(z3, 1),
  (z3 + 1, 1),
  (2*z3 + 1, 2),
  (z3^2 + z3, 2),
  (2*z3^2 + 1, 1),
  (2*z3^2 + z3, 1)],
 [(z3^2, 1),
  (z3^2 + 1, 1),
  (z3^2 + z3, 2),
  (z3^2 + 2*z3 + 1, 2),
  (2*z3^2 + z3, 2),
  (2*z3^2 + z3 + 1, 1)]]

In [185]:
L4bis

[[(z3^2, 2),
  (z3^2 + 1, 2),
  (z3^2 + z3 + 1, 1),
  (2*z3^2 + 1, 2),
  (2*z3^2 + z3, 1),
  (2*z3^2 + 2*z3 + 1, 2)],
 [(z3^2 + z3, 1),
  (z3^2 + z3 + 1, 2),
  (z3^2 + 2*z3 + 1, 1),
  (2*z3^2 + 1, 1),
  (2*z3^2 + z3 + 1, 2),
  (2*z3^2 + 2*z3 + 1, 1)],
 [(z3, 1),
  (z3 + 1, 1),
  (2*z3 + 1, 2),
  (z3^2 + z3, 2),
  (2*z3^2 + 1, 1),
  (2*z3^2 + z3, 1)],
 [(z3^2, 1),
  (z3^2 + 1, 1),
  (z3^2 + z3, 2),
  (z3^2 + 2*z3 + 1, 2),
  (2*z3^2 + z3, 2),
  (2*z3^2 + z3 + 1, 1)]]

In [24]:
d[2*z3+1]

[0 0 0]
[0 1 2]
[0 2 1]

In [150]:
k.vector_space()(2*z3 + 1)

(1, 2, 0)

In [25]:
P = k.modulus()

In [30]:
(z3+1).polynomial()(companion_matrix(k.modulus()))

[1 0 2]
[1 1 1]
[0 1 1]

In [31]:
companion_matrix?

In [33]:
z3.trace??

### Remark about this computation

We find only **one** solution, while the other method finds **two** solutions if you allow to search for solutions with 3 elements in each coordinate. If finds **four** solutions if you allow any type of solutions.

What is happening here is that one of the solution requires 3 terms in order to nullify a rank 2 matrix, and this is not seen by that method.

In [98]:
z3 = k.gen()

In [110]:
(T[0] - d[z3+1] - d[2*z3^2+1] + d[2*z3+1])

[0 0 0]
[0 0 0]
[0 0 0]

In [109]:
(T[0] - d[z3+1]).rank()

2

In [112]:
u1 = T[1] - d[z3+1] - d[2*z3+1]
u1

[0 1 0]
[1 1 1]
[0 1 0]

In [113]:
u1.rank()

2

In [114]:
(u1 - d[z3])

[0 1 0]
[1 0 1]
[0 1 0]

In [121]:
(u1 - d[z3]).rank()

2

In [126]:
(u1 - 2*d[z3^2+z3]).rank()

2

In [129]:
(u1 - d[z3^2+z3]).rank()

1

In [128]:
(u1 - d[2*z3^2+z3]).rank()

2

In [131]:
(u1 - d[z3] - d[2*z3^2+z3]).rank()

1

In [133]:
(u1 - d[z3] - d[2*z3^2+z3] - 2*d[z3^2+z3]).rank()

0

### Matrices with $\mathbb F_{3^4}$

In [224]:
p, n = 3, 4
k = GF(p^n)
S = MatrixSpace(GF(p), n, n)
d = make_dict_global_mat(k, S)
vectors = make_vectors(k)
T = multiplication_formula_mat(k, S)

In [202]:
L = []
%time tri_symmetric_search_mat(T, k, d, L)
len(L)

CPU times: user 172 ms, sys: 12.9 ms, total: 185 ms
Wall time: 168 ms


6

In [43]:
L2 = []
%time tri_symmetric_search_mat3(T, k, d, L2)
len(L2)

CPU times: user 221 ms, sys: 9.58 ms, total: 231 ms
Wall time: 216 ms


6

In [203]:
for l in L:
    print len(l)

9
10
10
9
10
10


In [240]:
L1, L3, L4, L4bis = test_algs(T, k, d, counts = [1, 1, 0, 0], bound = 9)

*** Algo 1 ***
CPU times: user 199 ms, sys: 12.2 ms, total: 211 ms
Wall time: 195 ms
*** Algo 3 ***
CPU times: user 420 ms, sys: 9.26 ms, total: 429 ms
Wall time: 394 ms
*** Algo 4 ***
CPU times: user 511 ms, sys: 16 ms, total: 527 ms
Wall time: 478 ms
*** Algo 4bis ***
CPU times: user 485 ms, sys: 16.2 ms, total: 502 ms
Wall time: 458 ms


In [241]:
len(L3), len(L4), L4 == L4bis

(8, 10, True)

In [239]:
for l in L: # L here was the one obtained in 3 min via the previous algorithm that is not based on matrices
    print l in L3

True
True
True
True
True


### Matrices with $\mathbb F_{3^5}$

In [242]:
p, n = 3, 5
k = GF(p^n)
S = MatrixSpace(GF(p), n, n)
d = make_dict_global_mat(k, S)
vectors = make_vectors(k)
T = multiplication_formula_mat(k, S)

In [45]:
L = []
%time tri_symmetric_search_mat(T, k, d, L, bound = 12)
len(L)

CPU times: user 2.43 s, sys: 117 ms, total: 2.54 s
Wall time: 2.42 s


4

In [46]:
L2 = []
%time tri_symmetric_search_mat3(T, k, d, L2, bound = 12)
len(L2)

CPU times: user 2.34 s, sys: 78 ms, total: 2.42 s
Wall time: 2.31 s


4

In [70]:
min([len(l) for l in L])

12

In [71]:
max([len(l) for l in L])

15

In [249]:
L1, L3, L4, L4bis = test_algs(T, k, d, counts = [0, 1, 1, 0, 0], bound = 11)

*** Algo 1 ***
CPU times: user 1.75 s, sys: 52.5 ms, total: 1.8 s
Wall time: 1.68 s
*** Algo 3 ***
CPU times: user 10.5 s, sys: 166 ms, total: 10.6 s
Wall time: 10.4 s
*** Algo 4 ***
CPU times: user 11.7 s, sys: 194 ms, total: 11.9 s
Wall time: 11.7 s
*** Algo 4bis ***
CPU times: user 12.5 s, sys: 165 ms, total: 12.7 s
Wall time: 12.4 s


In [250]:
len(L1), len(L3), len(L4), len(L4bis)

(0, 0, 0, 0)

In [263]:
L4 = []
%time tri_symmetric_search_mat4(T, k, d, L4, counts = [1, 2, 2, 2, 2], bound = 11)
len(L4)

CPU times: user 8min 56s, sys: 3.03 s, total: 8min 59s
Wall time: 8min 56s


0

### Note
The frobenius-fixed search finds a decomposition of length 11.

### Matrices with $\mathbb F_{3^6}$

In [17]:
p, n = 3, 6
k = GF(p^n)
S = MatrixSpace(GF(p), n, n)
d = make_dict_global_mat(k, S)
T = multiplication_formula_mat(k, S)

In [102]:
L = []
%time tri_symmetric_search_mat(T, k, d, L, bound = 6)
len(L)

CPU times: user 8.79 s, sys: 47.5 ms, total: 8.84 s
Wall time: 8.78 s


0

In [123]:
L2 = []
%time tri_symmetric_search_mat2(T, k, d, L2, bound = 6)
len(L2)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


KeyboardInterrupt: 

In [109]:
s1, s2 = S.random_element(), S.random_element()
%time (s1-s2).rank()

CPU times: user 94 µs, sys: 1 µs, total: 95 µs
Wall time: 93 µs


5

CPU times: user 122 µs, sys: 3 µs, total: 125 µs
Wall time: 130 µs


6

In [16]:
QuadraticForm?

In [19]:
q = QuadraticForm(ZZ, 2, [2, 1, 1])
q

Quadratic form in 2 variables over Integer Ring with coefficients: 
[ 2 1 ]
[ * 1 ]

In [20]:
q.is_definite()

True

In [21]:
G = q.automorphism_group()

In [22]:
G

Matrix group over Rational Field with 2 generators (
[-1  0]  [-1  0]
[ 0 -1], [ 1  1]
)

In [81]:
G.order()

4

In [59]:
g0, g1, g2 = G.gens()

In [60]:
g1^2

[1 0]
[0 1]

In [62]:
g2

[ 1 -4]
[ 0 -1]

In [65]:
# Other tests

"""
Return a random rank one element in `matrix_space`.
"""
def random_rank_one_element(matrix_space):
    dimension = matrix_space.ncols()
    base_ring = matrix_space.base_ring()
    vector_space = MatrixSpace(k, dimension, 1)
    y = vector_space.random_element()
    while y.is_zero():
        y = vector_space.random_element()
        
    return matrix_space(y*y.transpose())

In [207]:
p, n = 3, 3
k = GF(p)
R.<t> = k['t']
S = MatrixSpace(R, n, n)
U = MatrixSpace(k, n, n)

In [211]:
m = U.random_element()
while m.rank() < n:
    m = U.random_element()
x = random_rank_one_element(U)
y = random_rank_one_element(U)
while x == y:
    y = random_rank_one_element(U)
f = m - t*x
P = f.det()
P

1

In [212]:
m

[0 2 0]
[2 1 0]
[0 0 2]

In [213]:
x

[0 0 0]
[0 1 0]
[0 0 0]

In [161]:
f(1, 2).rank()

5

In [162]:
P.


Multivariate Polynomial Ring in t, u over Finite Field of size 3

In [166]:
m.rank()

4

In [178]:
len(m.minors(1))

25

In [19]:
U.bas

3

In [20]:
T = MatrixSpace(k, 3, 1)

In [39]:
y = T.random_element(nonzero=True)

In [44]:
y*y.transpose()

[1 1 2]
[1 1 2]
[2 2 1]

In [48]:
x = random_rank_one_element(U); x

[1 1 2]
[1 1 2]
[2 2 1]

In [50]:
t = R.gen()

In [51]:
S(m-t*x)

[    2*t 2*t + 2   t + 2]
[2*t + 1 2*t + 1   t + 2]
[      t   t + 2 2*t + 1]

In [57]:
f = m-t*x; f

[    2*t 2*t + 2   t + 2]
[2*t + 1 2*t + 1   t + 2]
[      t   t + 2 2*t + 1]

In [53]:
m.parent()

Full MatrixSpace of 3 by 3 dense matrices over Finite Field of size 3

In [54]:
x.parent()

Full MatrixSpace of 3 by 3 dense matrices over Finite Field of size 3

In [55]:
t.parent()

Univariate Polynomial Ring in t over Finite Field of size 3

In [58]:
f.parent() == S

True

In [59]:
f.det()

t + 2

In [60]:
f(1)

[2 1 0]
[0 0 0]
[1 0 0]

In [64]:
f(1).parent()

False

In [183]:
numerical_approx(log(binomial(160, 5), 2))

29.6117241704942

In [14]:
# Timings tests
k.bas

In [41]:
T = trace_form(k)

In [44]:
L = T*multiplication_matrix(z3+1)

In [47]:
(k^3)(L.list())

(0, 2, 2)

In [53]:
corresponding_vector(z3+1)

(0, 2, 2)

In [71]:
k

Finite Field in z5 of size 3^5

In [72]:
vectors = dict()
for a in k:
    vectors[a] = corresponding_vector(a)

In [74]:
vectors

{0: (0, 0, 0, 0, 0),
 1: (2, 0, 0, 0, 1),
 2: (1, 0, 0, 0, 2),
 z5: (0, 0, 0, 1, 1),
 z5 + 1: (2, 0, 0, 1, 2),
 z5 + 2: (1, 0, 0, 1, 0),
 2*z5: (0, 0, 0, 2, 2),
 2*z5 + 1: (2, 0, 0, 2, 0),
 2*z5 + 2: (1, 0, 0, 2, 1),
 z5^2: (0, 0, 1, 1, 0),
 z5^2 + 1: (2, 0, 1, 1, 1),
 z5^2 + 2: (1, 0, 1, 1, 2),
 z5^2 + z5: (0, 0, 1, 2, 1),
 z5^2 + z5 + 1: (2, 0, 1, 2, 2),
 z5^2 + z5 + 2: (1, 0, 1, 2, 0),
 z5^2 + 2*z5: (0, 0, 1, 0, 2),
 z5^2 + 2*z5 + 1: (2, 0, 1, 0, 0),
 z5^2 + 2*z5 + 2: (1, 0, 1, 0, 1),
 2*z5^2: (0, 0, 2, 2, 0),
 2*z5^2 + 1: (2, 0, 2, 2, 1),
 2*z5^2 + 2: (1, 0, 2, 2, 2),
 2*z5^2 + z5: (0, 0, 2, 0, 1),
 2*z5^2 + z5 + 1: (2, 0, 2, 0, 2),
 2*z5^2 + z5 + 2: (1, 0, 2, 0, 0),
 2*z5^2 + 2*z5: (0, 0, 2, 1, 2),
 2*z5^2 + 2*z5 + 1: (2, 0, 2, 1, 0),
 2*z5^2 + 2*z5 + 2: (1, 0, 2, 1, 1),
 z5^3: (0, 1, 1, 0, 0),
 z5^3 + 1: (2, 1, 1, 0, 1),
 z5^3 + 2: (1, 1, 1, 0, 2),
 z5^3 + z5: (0, 1, 1, 1, 1),
 z5^3 + z5 + 1: (2, 1, 1, 1, 2),
 z5^3 + z5 + 2: (1, 1, 1, 1, 0),
 z5^3 + 2*z5: (0, 1, 1, 2, 2),
 z5^3 +

In [96]:
CPT2 = 0
def toto():
    global CPT2
    CPT2 += 1

In [99]:
toto()

In [100]:
CPT2

2

Une fonction qui s'appelle seule avec un compteur qui diminue et qui lance la recherche de décomposition minimale quand le compteur est à zéro sur un ensemble d'éléments calculés par la fonction

In [35]:
def compute_search_set(target, fields_elems, count, dictionnary, global_set, local_set = [], index = 0):
    if count == 0:
        global_set.append((local_set, index, target))
    else:
        Field = target.base_ring()
        nonZeroElems = list(Field)[1:]
        for j in range(index, len(fields_elems)):
            elem = fields_elems[j]
            for x in nonZeroElems:
                new_target = target-x*dictionnary[elem]
                compute_search_set(new_target, fields_elems, count-1, dictionnary, global_set, local_set+[(elem, x)], j+1)

def find_decomposition_mat3(t, B, d, L, begin = [], G = []):
    if t.rank() == 0:
        L.append(begin+G)
    else:
        r = t.rank()
        Field = t.base_ring()
        nonZeroElems = list(Field)[1:]
        elems = []
        for b in B:
            for x in nonZeroElems:
                if (t - x*d[b]).rank() < r:
                    elems.append((b, x))
                    break
        for j in range(len(elems)):
            y = elems[j]
            B2 = [elems[i][0] for i in range(j+1, len(elems))]
            find_decomposition_mat3(t-y[1]*d[y[0]], B2, d, L, begin, copy(G+[y]))

def tri_symmetric_search_mat3(T, k, d, Lglob, counts = None, bound = Infinity, j = 0, Lloc = []):
    if counts == None:
        counts = len(T)*[0]
    
    if finished(T):
        Lloc.sort()
        Lglob.append(Lloc)
    elif T[j].rank() <= bound:
        
        M = []
        search_set = []
        fields_elems = fields_elements(k, j)
        count = counts[j]
        compute_search_set(T[j], fields_elems, count, d, search_set)

        for t in search_set:
            find_decomposition_mat3(t[2], fields_elems[t[1]:], d, M, t[0])
        
        for m in M:
            T2 = copy(T)
            T2[j] = 0
            Lloc2 = Lloc+m
            for x in m:
                P = x[0].polynomial()
                for l in range(j+1, P.degree()+1):
                    T2[l] = T2[l] - P[l]*x[1]*d[x[0]]
            
            tri_symmetric_search_mat3(T2, k, d, Lglob, counts, bound-len(m), j+1, Lloc2)

In [206]:
def find_decomposition_mat4(t, B, d, L, count, bound, G = []):
    r = t.rank()
    if r <= bound:
        if r == 0:
            L.append(G)
        elif count == 0:
            Field = t.base_ring()
            #nonZeroElems = [x for x in Field if x != 0]
            nonZeroElems = list(Field)[1:]
            elems = []
            for b in B:
                for x in nonZeroElems:
                    if (t - x*d[b]).rank() < r:
                        elems.append((b, x))
                        break
            for j in range(len(elems)):
                y = elems[j]
                B2 = [elems[i][0] for i in range(j+1, len(elems))]
                find_decomposition_mat4(t-y[1]*d[y[0]], B2, d, L, count, bound-1, copy(G+[y]))
        else:
            Field = t.base_ring()
            #nonZeroElems = [x for x in Field if x != 0]
            nonZeroElems = list(Field)[1:]
            for j in range(len(B)):
                b = B[j]
                B2 = B[j+1:]
                for x in nonZeroElems:
                    t2 = t - x*d[b]
                    r2 = t2.rank()
                    if r2 < r:                    
                        find_decomposition_mat4(t2, B2, d, L, count, bound-1, copy(G+[(b, x)]))
                    elif r2 == r: #else what happens if we allow the rank to rise? ie r2 > r?
                        find_decomposition_mat4(t2, B2, d, L, count-1, bound-1, copy(G+[(b, x)]))

"""
Compute a list `Lglob` of tri-symmetric decompositions, where for each coordinate
the search is minimal.  
"""
def tri_symmetric_search_mat4(T, k, d, Lglob, counts = None, bound = Infinity, j = 0, Lloc = []):
    if counts == None:
        counts = len(T)*[0]
        
    if finished(T):
        Lloc.sort()
        Lglob.append(Lloc)
    elif T[j].rank() <= bound:
        M = []
        count = counts[j] # min(count[j], bound) smth like that?
        find_decomposition_mat4(T[j], fields_elements(k, j), d, M, count, bound)

        for m in M:
            T2 = copy(T)
            T2[j] = 0
            Lloc2 = Lloc+m
            for x in m:
                P = x[0].polynomial()
                for l in range(j+1, P.degree()+1):
                    T2[l] = T2[l] - P[l]*x[1]*d[x[0]]
            
            tri_symmetric_search_mat4(T2, k, d, Lglob, counts, bound-len(m), j+1, Lloc2)
            
def find_decomposition_mat4bis(t, B, d, L, count, bound, G = [], index = 0):
    r = t.rank()
    if r <= bound:
        if r == 0:
            L.append(G)
        elif count == 0:
            Field = t.base_ring()
            #nonZeroElems = [x for x in Field if x != 0]
            nonZeroElems = list(Field)[1:]
            elems = []
            for j in range(index, len(B)):
                b = B[j]
                for x in nonZeroElems:
                    if (t - x*d[b]).rank() < r:
                        elems.append((b, x))
                        break
            for j in range(len(elems)):
                y = elems[j]
                B2 = [elems[i][0] for i in range(j+1, len(elems))]
                find_decomposition_mat4bis(t-y[1]*d[y[0]], B2, d, L, count, bound-1, copy(G+[y]))
        else:
            Field = t.base_ring()
            #nonZeroElems = [x for x in Field if x != 0]
            nonZeroElems = list(Field)[1:]
            for j in range(index, len(B)):
                b = B[j]
                for x in nonZeroElems:
                    t2 = t - x*d[b]
                    r2 = t2.rank()
                    if r2 < r:                    
                        find_decomposition_mat4bis(t2, B, d, L, count, bound-1, copy(G+[(b, x)]), j+1)
                    elif r2 == r: #else what happens if we allow the rank to rise? ie r2 > r?
                        find_decomposition_mat4bis(t2, B, d, L, count-1, bound-1, copy(G+[(b, x)]), j+1)

"""
Compute a list `Lglob` of tri-symmetric decompositions, where for each coordinate
the search is minimal.  
"""
def tri_symmetric_search_mat4bis(T, k, d, Lglob, counts = None, bound = Infinity, j = 0, Lloc = []):
    if counts == None:
        counts = len(T)*[0]
        
    if finished(T):
        Lloc.sort()
        Lglob.append(Lloc)
    elif T[j].rank() <= bound:
        M = []
        count = counts[j] # min(count[j], bound) smth like that?
        find_decomposition_mat4bis(T[j], fields_elements(k, j), d, M, count, bound)

        for m in M:
            T2 = copy(T)
            T2[j] = 0
            Lloc2 = Lloc+m
            for x in m:
                P = x[0].polynomial()
                for l in range(j+1, P.degree()+1):
                    T2[l] = T2[l] - P[l]*x[1]*d[x[0]]
            
            tri_symmetric_search_mat4bis(T2, k, d, Lglob, counts, bound-len(m), j+1, Lloc2)
            
def test_algs(T, k, d, counts = None, bound = Infinity):
    
    print "*** Algo 1 ***"
    L = []
    %time tri_symmetric_search_mat(T, k, d, L, bound)
    
    print "*** Algo 3 ***"
    L3 = []
    %time tri_symmetric_search_mat3(T, k, d, L3, counts, bound)
    
    print "*** Algo 4 ***"
    L4 = []
    %time tri_symmetric_search_mat4(T, k, d, L4, counts, bound)
    
    print "*** Algo 4bis ***"
    L4bis = []
    %time tri_symmetric_search_mat4bis(T, k, d, L4bis, counts, bound)
    
    return L, L3, L4, L4bis

In [118]:
L2 = []
%time tri_symmetric_search_mat3(T, k, d, L2)
L2

CPU times: user 13.7 ms, sys: 115 µs, total: 13.8 ms
Wall time: 12.4 ms


[[(z3^2, 1),
  (z3^2 + 1, 1),
  (z3^2 + z3, 2),
  (z3^2 + 2*z3 + 1, 2),
  (2*z3^2 + z3, 2),
  (2*z3^2 + z3 + 1, 1)]]

In [119]:
L4 = []
%time tri_symmetric_search_mat4(T, k, d, L4, bound = 6)
L4

CPU times: user 15.9 ms, sys: 300 µs, total: 16.2 ms
Wall time: 14.2 ms


[[(z3^2, 1),
  (z3^2 + 1, 1),
  (z3^2 + z3, 2),
  (z3^2 + 2*z3 + 1, 2),
  (2*z3^2 + z3, 2),
  (2*z3^2 + z3 + 1, 1)]]

In [18]:
a[2:]

[5, 6, 7, 8, 9, 10]

In [70]:
def toto(L, j = 0):
    if j < len(L):
        if L[j] == 0:
            L[j] = 777
            toto(L, j+1)
        else:
            toto(L, j+1)
    

def toto2(L):
    if len(L) > 0:
        if L[0] == 0:
            L[0] = 777
            toto(L[1:])
        else:
            toto(L[1:])

In [95]:
a =  200*range(3)

In [96]:
%time toto(a)

CPU times: user 685 µs, sys: 33 µs, total: 718 µs
Wall time: 523 µs


In [97]:
b =  200*range(3)

In [98]:
%time toto2(b)

CPU times: user 1.28 ms, sys: 0 ns, total: 1.28 ms
Wall time: 1.07 ms


In [138]:
[[]] + range(3)

[[], 0, 1, 2]

In [36]:
b[1] = 777

In [37]:
a, b

([0, 1, 2, 3], [1, 777, 3])

In [111]:
k = GF(3^7)

In [112]:
list(k)[0]

0

In [139]:
def toto():
    global yy
    yy = 33

In [140]:
toto()

In [141]:
yy

33

In [15]:
k = GF(3^5)

In [16]:
x = k.gen()

[[z5^2 + z5 + 1,
  z5^3 + z5^2 + 2*z5 + 1,
  2*z5^4 + 2*z5^3 + z5^2,
  2*z5^3 + z5 + 1,
  2*z5^4 + 2*z5^3 + z5 + 1],
 [z5^4 + 2*z5^3 + 2*z5 + 1,
  z5^4 + 2*z5^2 + z5 + 1,
  z5^4 + 2*z5^3 + z5 + 1,
  z5^4 + z5^3 + 2*z5^2 + z5 + 1,
  z5^3 + z5],
 [2*z5^3 + 2*z5^2 + z5 + 1,
  2*z5^4 + 2*z5^3 + z5^2 + 1,
  z5^3 + 2*z5 + 1,
  2*z5^4 + 2*z5^3 + z5,
  2*z5^2 + 2*z5 + 1],
 [2*z5^4 + z5^3 + z5^2 + 2*z5 + 1,
  z5^4 + z5^3,
  2*z5^3 + 2*z5^2 + 2*z5 + 1,
  2*z5^4 + z5^3 + z5^2 + 1,
  2*z5^4 + z5^3],
 [z5^4 + z5^3 + 2*z5^2 + 1,
  2*z5^3 + z5,
  z5^4 + z5^3 + 2*z5 + 1,
  z5^2 + z5,
  2*z5^3 + 2*z5^2 + z5],
 [z5^4 + z5^3 + z5 + 1,
  2*z5^3 + z5^2 + z5,
  z5^4 + z5^3 + z5^2 + z5 + 1,
  z5^3 + z5^2,
  z5^4 + 2*z5^2 + 1],
 [2*z5^3 + z5^2 + z5 + 1,
  2*z5^4 + 2*z5^3 + 2*z5^2 + 2*z5 + 1,
  2*z5^3 + 2*z5^2 + 1,
  2*z5^4 + z5^2 + 1,
  2*z5^4 + 2*z5^3 + 2*z5 + 1],
 [z5^4 + z5^3 + z5^2 + 2*z5 + 1,
  z5^2,
  2*z5^2 + z5,
  z5^3 + 2*z5^2 + z5,
  z5^4 + 2*z5^3 + z5^2 + z5 + 1],
 [2*z5^4 + z5 + 1,
  2*z5^4 + 2*z5

In [38]:
y = k.random_element()

In [39]:
d[y]

[0 0 0 0 0 0]
[0 1 1 0 2 1]
[0 1 1 0 2 1]
[0 0 0 0 0 0]
[0 2 2 0 1 2]
[0 1 1 0 2 1]

In [40]:
d[y.frobenius()]

[0 0 0 0 0 0]
[0 1 2 2 0 0]
[0 2 1 1 0 0]
[0 2 1 1 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]

In [24]:
y

2*z6^5 + 2*z6^3 + z6^2 + 1